In [18]:
# 라이브러리 임포트

import numpy as np
import pandas as pd

import re
import folium

import requests
import json

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import dash_table_experiments as dt

import warnings
warnings.filterwarnings(action = 'ignore')

from bs4 import BeautifulSoup


# 사용자 모듈

from make_lonlat import lonlat
from change_lonlat import findlonlat

# 지도 시각화
from collections import OrderedDict
import geojson
import os
import pickle

In [19]:
traffic = ['busstation','taxistop','subway']
education = ['academy','elementary','high','kindergarden','lifelongedu','middle']
welfare = ['childcarelibrary','childrencenter','library','infant','park',
           'seniorcenter','oldwelfare','hospital','publicparking','trashcan']
safety = ['CCTV','crosswalk','emergencyroom','femalezone','firestation','police','chiancenter']

score_list = ['traffic_score', 'education_score', 'welfare_score', 'safety_score', 'total_score']

df = pd.read_csv('scoredf.csv')

In [21]:
# 시각화를 위한 데이터프레임 변환

transpose_df = df[['lonlat',
                   'traffic_score',
                   'education_score',
                   'welfare_score',
                   'safety_score',
                   'total_score']].set_index('lonlat').transpose()

transpose_df.head()

lonlat,lon15_lat1,lon16_lat1,lon17_lat1,lon18_lat1,lon15_lat2,lon16_lat2,lon17_lat2,lon18_lat2,lon14_lat3,lon15_lat3,...,lon6_lat19,lon11_lat19,lon6_lat20,lon10_lat20,lon6_lat21,lon9_lat21,lon6_lat22,lon8_lat22,lon6_lat23,lon7_lat23
traffic_score,0.000000,0.177778,0.133333,0.044444,0.266667,0.222222,0.266667,0.355556,0.133333,0.177778,...,0.000000,0.142857,0.044444,0.187302,0.00000,0.0,0.142857,0.142857,0.142857,0.142857
education_score,0.186420,0.219753,0.256790,0.132716,0.270194,0.841623,1.161993,0.804145,0.614638,0.581305,...,0.083774,0.024691,0.071429,0.083774,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
welfare_score,0.466667,0.450000,0.300000,0.189716,0.797074,0.998227,0.481649,0.458245,0.853812,0.926064,...,0.000000,0.300000,0.062500,0.300000,0.02500,0.0,0.050000,0.025000,0.025000,0.025000
safety_score,0.368012,0.326087,0.416149,0.142857,0.423913,0.364907,0.409938,0.358696,0.395963,0.420807,...,0.024845,0.357143,0.018634,0.214286,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
total_score,0.255275,0.293404,0.276568,0.127433,0.439462,0.606745,0.580062,0.494160,0.499437,0.526489,...,0.027155,0.206173,0.049252,0.196340,0.00625,0.0,0.048214,0.041964,0.041964,0.041964


In [15]:
# 지도 시각화 - 카토그램
import pickle
with open('final_dict.pkl','rb') as f:
    final_dict = pickle.load(f)
    
    
coor_data = OrderedDict()

coor_data["type"] = "FeatureCollection"
coor_data["name"] = "강서구_25*25분할"

features=[]
for k,v in final_dict.items():
    features.append({ "type": "Feature", "properties": { "name": k },
    "geometry": { "type": "MultiPolygon", "coordinates": [ [v] ] } })

coor_data["features"] = features

with open('square.json','w',encoding="utf-8") as make_file:
    json.dump(coor_data,make_file,ensure_ascii=False)

with open('square0.geojson','w',encoding="utf-8") as make_file:
    geojson.dump(coor_data,make_file,ensure_ascii=False)



square_gj = json.load(open('square0.geojson', encoding = 'utf-8'))

scoredf = pd.read_csv('scoredf.csv', encoding = 'cp949')

scoredf = scoredf.rename(columns={'lonlat':'name'})

fig = px.choropleth_mapbox(scoredf, geojson=square_gj, locations='name', color='total_score',
                           color_continuous_scale='speed',
                           range_color=(0,1),
                           mapbox_style="carto-positron",
                           zoom=12.2,center={"lat":37.559819, "lon": 126.825895 },
                           featureidkey='properties.name',
                           opacity=0.6)
fig.update_layout(
width=1100,
height=900)


In [16]:
# 시각화 - 마커표시

df = pd.read_csv('lonlatdata.csv',encoding='cp949')

df_traffic= df.loc[df['category'] == 'traffic']
df_education= df.loc[df['category'] == 'education']
df_welfare= df.loc[df['category'] == 'welfare']
df_safety=df.loc[df['category'] == 'safety']

df_traffic.type

traffic_lat=df_traffic.latitude
traffic_lon=df_traffic.longitude
education_lat=df_education.latitude
education_lon=df_education.longitude
welfare_lat=df_welfare.latitude
welfare_lon=df_welfare.longitude
safety_lat=df_safety.latitude
safety_lon=df_safety.longitude

traffic_locations_name= list(df_traffic["type"])
len(traffic_locations_name)

fig_traffic=px.scatter_mapbox(df_traffic,
                               lat=traffic_lat,
                               lon=traffic_lon,
                               hover_data=[traffic_locations_name],
                               color_discrete_sequence=["red"],zoom=15,
                               center={"lat":37.559819, "lon": 126.825895 }
                               )
fig_traffic.update_layout(mapbox_style="open-street-map")
fig_traffic.show()

education_locations_name=list(df_education["type"])
fig_education=px.scatter_mapbox(df_education,
                               lat=education_lat,
                               lon=education_lon,
                               hover_data=[education_locations_name],
                               color_discrete_sequence=["yellow"],zoom=15,
                               center={"lat":37.559819, "lon": 126.825895 }
                               )
fig_education.update_layout(mapbox_style="open-street-map")
fig_education.show()

welfare_locations_name=list(df_welfare["type"])
fig_welfare=px.scatter_mapbox(df_welfare,
                               lat=welfare_lat,
                               lon=welfare_lon,
                               hover_data=[welfare_locations_name],
                               color_discrete_sequence=["orange"],zoom=15,
                               center={"lat":37.559819, "lon": 126.825895 }
                               )
fig_welfare.update_layout(mapbox_style="open-street-map")
fig_welfare.show()

safety_locations_name=list(df_safety["type"])
fig_safety=px.scatter_mapbox(df_safety,
                               lat=safety_lat,
                               lon=safety_lon,
                               hover_data=[safety_locations_name],
                               color_discrete_sequence=["green"],zoom=15,
                               center={"lat":37.559819, "lon": 126.825895 }
                               )
fig_safety.update_layout(mapbox_style="open-street-map")
fig_safety.show()

In [7]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.GRID])

app.layout = html.Div([
    html.Div(
        children = [
            html.H1('강서구 도시증진 맞춤점수 지표', ),
            html.H3('해당 서비스는 강서구의 발전을 위해 고안된 것입니다. 주소를 입력하면 해당 구역을 교육/교통/안전/복지의 4가지 항목으로 나누어 세부적인 점수 비교를 통해 지역 특성을 파악할 수 있습니다.'),
            html.Br(),
            # 주소를 입력하는 텍스트창 생성
            html.Label('주소를 입력하세요 :   ', style = {'font-size': '115%'}),
            dcc.Input(
                type='text',
                id = 'text-input', style = {'font-size': '115%'}),
            # 검색을 위한 버튼
            html.Button(id='submit-button-state', n_clicks=0, children='확인', style = {'background-color':'#A7C85A','font-size': '115%'}),
            dcc.Graph(figure = fig) # 카토그램
        ],
        style = {'padding': 30, 'flex': 1, 'textAlign': 'center', 'font-size': '120%'} # CSS
    ),
    html.Div(
        children = [
            html.H3('카테고리점수: 전체 지역 평균과 해당 지역과의 차이, 카테고리끼리의 차이를 볼 수 있습니다.'),
            # 그래프 1,2 - 바플랏1과 파이차트
            dbc.Row([   
                dbc.Col(dcc.Graph(id = 'graph1')),
                dbc.Col(dcc.Graph(id = 'pie-graph'))
            ]),
            # 카테고리를 고르는 RadioItems
            dcc.RadioItems(['traffic','education','welfare','safety'], id = 'radioitem-input1', style = {'border-style': 'solid', 'border-width': 'thin'}),
            html.Br(),
            html.H3('시설물점수: 특정 카테고리에 포함된 시설물에 대한 세부점수를 확인할 수 있습니다.'),
            # 그래프3 - 바플랏2
            dcc.Graph(id = 'graph2')
        ],
        style = {'padding': 10, 'flex': 1, 'textAlign': 'left', 'font-size': '150%'}
    )
],
style={'display': 'flex', 'flex-direction': 'row'})

# Callback1 - 왼쪽 위 바플랏 콜백
@app.callback(
    Output('graph1', 'figure'),
    Input('submit-button-state', 'n_clicks'),
    State('text-input', 'value'))
def update_graph1(n_clicks,address):
    lat, lon = findlonlat(address)
    
    colors = ['#496449',] * 5 # 초록
    colors[-1] = '#FE421A' # 다홍

    fig = go.Figure(data=[
        go.Bar(name='Current location', x=score_list, y=transpose_df[lonlat(lat,lon)],  marker_color=colors),
        go.Bar(name='All', x = score_list, y = transpose_df.mean(axis = 1), marker_color = '#F49B24') # 노랑
    ])
    fig.update_layout(plot_bgcolor="white")
    fig.update_xaxes(linecolor='black', gridcolor='gray',mirror=True)
    fig.update_yaxes(linecolor='black', gridcolor='gray',mirror=True)
    return fig

# Callback2 - 오른쪽 위 파이차트 콜백
@app.callback(
    Output('pie-graph', 'figure'),
    Input('submit-button-state', 'n_clicks'),
    State('text-input', 'value'))
def update_piegraph(n_clicks,address):
    lat, lon = findlonlat(address)
    
    colors = ['#F7D480', '#F0B02D', '#65802A', '#E7C9A5']
    
    fig = go.Figure(data=[go.Pie(labels=transpose_df.index[:-1], values=transpose_df[lonlat(lat,lon)][:-1])])
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors))
    return fig

# Callback3 - 아래 바플랏 콜백
@app.callback(
    Output('graph2', 'figure'),
    Input('submit-button-state', 'n_clicks'),
    Input('radioitem-input1', 'value'),
    State('text-input', 'value'))
def update_graph2(n_clicks,category,address):
    lat, lon = findlonlat(address)
    
    if category == 'traffic':
        transpose_df2 = (df[['lonlat'] + traffic]).set_index('lonlat').transpose()
    elif category == 'education':
        transpose_df2 = (df[['lonlat'] + education]).set_index('lonlat').transpose()
    elif category == 'welfare':
        transpose_df2 = (df[['lonlat'] + welfare]).set_index('lonlat').transpose()
    elif category == 'safety':
        transpose_df2 = (df[['lonlat'] + safety]).set_index('lonlat').transpose()
        
    y = transpose_df2[lonlat(lat,lon)] - transpose_df2.mean(axis = 1)
    
    fig = go.Figure(data=[
       go.Bar(name='graph2', x=transpose_df2.index, y=y, marker_color = ['#496449' if i >= 0 else '#F49B24' for i in y])],
       layout = {
       'xaxis': {'title': category },
       'yaxis': {'title': '강서구 평균점수와의 차이', 'range': [-0.2, 0.2]}})
    fig.update_layout(plot_bgcolor="white")
    fig.update_xaxes(linecolor='black', gridcolor='gray',mirror=True)
    fig.update_yaxes(linecolor='black', gridcolor='gray',mirror=True)
    return fig

if __name__ == '__main__':
    app.run_server(host='127.0.0.1',port='7777')

Dash is running on http://127.0.0.1:7777/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:7777/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2022 23:27:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:27:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:27:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:27:02] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:27:02] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispat

127.0.0.1 - - [26/Mar/2022 23:27:02] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispat

...

127.0.0.1 - - [26/Mar/2022 23:35:12] "POST /_dash-update-component HTTP/1.1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispat

" 500 -
127.0.0.1 - - [26/Mar/2022 23:35:12] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Mar/2022 23:35:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:35:25] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\heejeong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\heejeong\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispat

127.0.0.1 - - [26/Mar/2022 23:35:25] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Mar/2022 23:35:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:35:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:35:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 23:35:41] "POST /_dash-update-component HTTP/1.1" 200 -
